### Model_Train_00

### 패키지 설치

In [1]:
!pip install facenet-pytorch

In [2]:
!pip install --upgrade torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 데이터 전처리

### 패키지

In [ ]:
import os
import zipfile
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm
from google.colab import drive

In [ ]:
# Step 1: Google Drive 마운트
drive.mount('/content/drive')

In [ ]:
# Step 2: 경로 설정
video_folder = '/content/drive/MyDrive/datasets/video'  # 비디오 zip 파일 폴더 경로 (변경 필요)
output_folder = '/content/drive/MyDrive/datasets/ouput'  # 얼굴 이미지 저장 폴더
zip_file_path = '/content/drive/MyDrive/datasets/train_sample_videos.zip'  # 비디오 zip 파일 경로


In [ ]:
# Step 3: zip 파일 압축 해제
if not os.path.exists(video_folder):
    os.makedirs(video_folder)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(video_folder)


In [ ]:
# Step 4: GPU 또는 CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

사용 중인 장치: cuda


In [ ]:
# Step 5: GPU를 사용한 MTCNN 초기화
mtcnn = MTCNN(keep_all=True, device=device)

/usr/local/lib/python3.10/dist-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/usr/local

In [ ]:
# Step 6: 추가 설정
frame_interval = 10  # 프레임 간격
resize_dim = (224, 224)  # 얼굴 이미지 크기

In [ ]:
# Step 7: 비디오 처리 함수
def process_video(video_path, output_folder, mtcnn, frame_interval, resize_dim):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_count = 0
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 간격에 따라 샘플링
        if frame_count % frame_interval == 0:
            # OpenCV BGR 이미지를 PIL RGB로 변환
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)

            # MTCNN을 통한 얼굴 감지
            boxes, _ = mtcnn.detect(pil_img)
            if boxes is not None:
                for i, box in enumerate(boxes):
                    # 얼굴 크롭 및 리사이즈
                    face = pil_img.crop((box[0], box[1], box[2], box[3])).resize(resize_dim)
                    face.save(os.path.join(output_folder, f"{video_name}_face_{extracted_count}.jpg"))
                    extracted_count += 1

        frame_count += 1

    cap.release()

In [ ]:
# Step 8: 모든 비디오 파일 처리
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]
os.makedirs(output_folder, exist_ok=True)

for video_file in tqdm(video_files, desc="비디오 처리 중"):
    video_path = os.path.join(video_folder, video_file)
    process_video(video_path, output_folder, mtcnn, frame_interval, resize_dim)
    print(f"{video_file} 처리 완료")

비디오 처리 중:   0%|          | 1/400 [00:12<1:22:29, 12.41s/it]

aagfhgtpmv.mp4 처리 완료


비디오 처리 중:   0%|          | 2/400 [00:20<1:05:11,  9.83s/it]

aapnvogymq.mp4 처리 완료


비디오 처리 중:   1%|          | 3/400 [00:29<1:02:35,  9.46s/it]

...

etohcvnzbj.mp4 처리 완료


비디오 처리 중: 100%|█████████▉| 399/400 [1:12:50<00:12, 12.94s/it]

eudeqjhdfd.mp4 처리 완료


비디오 처리 중: 100%|██████████| 400/400 [1:12:56<00:00, 10.94s/it]

eukvucdetx.mp4 처리 완료


## Model Train

### 패키지 설치

In [3]:
# EfficientNet-PyTorch 설치
!pip install efficientnet-pytorch

# pretrainedmodels 설치 (Xception 모델용)
!pip install pretrainedmodels


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=3a533d2849f3ec80a8f95994f7b1d089a7a1bb29328659187452e31fb1f01124
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=be422c3e770af1a945041a8d1088e0efc8d0900a0493eb3d68c859360cacfcaa
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built pretrainedmodels


In [4]:
import os
import ssl
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights
from efficientnet_pytorch import EfficientNet
from pretrainedmodels import xception
from PIL import Image
from tqdm import tqdm
from google.colab import drive

In [5]:
# SSL 인증서 검증 비활성화
ssl._create_default_https_context = ssl._create_unverified_context

class FaceForensicsDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.jpg', '.png'))]
        self.labels = [1 if "REAL" in img else 0 for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # NaN 검사
        if torch.isnan(image).any() or torch.isinf(image).any():
            print(f"Data issue in image {img_path}")
        return image, label

# 데이터 전처리 및 로더 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = FaceForensicsDataset(image_folder='/content/drive/MyDrive/datasets/ouput', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [6]:
# GPU 장치 설정 (Colab에서는 'cuda' 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 설정
xception_model = xception(num_classes=1000, pretrained='imagenet')
xception_model.last_linear = nn.Linear(xception_model.last_linear.in_features, 2)
xception_model = xception_model.to(device)

efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
efficientnet_model = efficientnet_model.to(device)

resnet_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)
resnet_model = resnet_model.to(device)  # 모델을 MPS에 올리기
# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = Adam(
    list(xception_model.parameters()) +
    list(efficientnet_model.parameters()) +
    list(resnet_model.parameters()),
    lr=1e-5  # 학습률을 낮추어 설정
)

# NaN 검사 함수
def check_nan(tensor, name=""):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}")
        return True
    return False

# 앙상블 예측 함수
def ensemble_predict(models, images):
    outputs = [model(images) for model in models]
    outputs = torch.stack(outputs).mean(dim=0)
    return outputs

# 학습 및 검증 함수 정의
def train_ensemble_model(models, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        for model in models:
            model.train()

        train_loss, train_correct = 0.0, 0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = ensemble_predict(models, images)

            # NaN 검사
            if check_nan(outputs, "outputs"):
                return

            loss = criterion(outputs, labels)
            if check_nan(loss, "loss"):
                return

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        train_acc = train_correct / len(train_dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")

        # Validation
        for model in models:
            model.eval()

        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                images, labels = images.to(device), labels.to(device)
                outputs = ensemble_predict(models, images)
                if check_nan(outputs, "validation outputs"):
                    return

                loss = criterion(outputs, labels)
                if check_nan(loss, "validation loss"):
                    return

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

        val_acc = val_correct / len(val_dataset)
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_acc:.4f}")




Using device: cuda


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth
100%|██████████| 87.4M/87.4M [04:17<00:00, 356kB/s]
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 102MB/s] 


Loaded pretrained weights for efficientnet-b0


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s]


In [7]:
# 학습 시작
models = [xception_model, efficientnet_model, resnet_model]
train_ensemble_model(models, criterion, optimizer, num_epochs=5)

Training Epoch 1/5: 100%|██████████| 719/719 [47:15<00:00,  3.94s/it]


Epoch [1/5], Loss: 0.0944, Accuracy: 0.9963


Validation Epoch 1/5: 100%|██████████| 180/180 [00:20<00:00,  8.95it/s]


Validation Loss: 0.0069, Validation Accuracy: 1.0000


Training Epoch 2/5: 100%|██████████| 719/719 [03:16<00:00,  3.65it/s]


...

Validation Loss: 0.0009, Validation Accuracy: 1.0000


Training Epoch 5/5: 100%|██████████| 719/719 [03:16<00:00,  3.66it/s]


Epoch [5/5], Loss: 0.0007, Accuracy: 1.0000


Validation Epoch 5/5: 100%|██████████| 180/180 [00:19<00:00,  9.01it/s]

Validation Loss: 0.0005, Validation Accuracy: 1.0000


In [8]:
# 모델 저장 경로 설정
output_dir = '/content/drive/MyDrive/datasets'
os.makedirs(output_dir, exist_ok=True)

# 모델 저장
torch.save(xception_model.state_dict(), os.path.join(output_dir, 'xception_model.pth'))
torch.save(efficientnet_model.state_dict(), os.path.join(output_dir, 'efficientnet_model.pth'))
torch.save(resnet_model.state_dict(), os.path.join(output_dir, 'resnet_model.pth'))

print("모델이 성공적으로 저장되었습니다.")

모델이 성공적으로 저장되었습니다.
